In [2]:
import requests
import os
import pandas as pd

TOKEN_URL = 'https://accounts.spotify.com/api/token'
BASE_URL = 'https://api.spotify.com/v1/'

# 1. Retrieve access token and call Spotify API

In [3]:
os.environ

environ{'USER': 'lucie_unix',
        'VSCODE_WSL_EXT_LOCATION': '/mnt/c/Users/Lucie/.vscode/extensions/ms-vscode-remote.remote-wsl-0.88.5',
        'WT_PROFILE_ID': '{51855cb2-8cce-5362-8f54-464b92b32386}',
        'SHLVL': '1',
        'HOME': '/home/lucie_unix',
        'OLDPWD': '/mnt/c/Users/Lucie/AppData/Local/Programs/Microsoft VS Code',
        'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus',
        'WSL_DISTRO_NAME': 'Ubuntu',
        'WAYLAND_DISPLAY': 'wayland-0',
        'LOGNAME': 'lucie_unix',
        'PULSE_SERVER': 'unix:/mnt/wslg/PulseServer',
        'WSL_INTEROP': '/run/WSL/42723_interop',
        'NAME': 'Code',
        '_': '/home/lucie_unix/.pyenv/versions/3.10.12/envs/Artefact/bin/python',
        'TERM': 'xterm-color',
        'PATH': '/home/lucie_unix/.pyenv/versions/Artefact/bin:/home/lucie_unix/.pyenv/versions/3.10.12/envs/Artefact/bin:/home/lucie_unix/.vscode-server/bin/e54c774e0add60467559eb0d1e229c6452cf8447/bin/remote-cli:/home/lucie_unix/.pye

In [4]:
## Envoyer une requête POST avec nos credentials 
CLIENT_ID=os.environ.get('CLIENT_ID')
CLIENT_SECRET=os.environ.get('CLIENT_SECRET')

data = {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
}

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

auth_response = requests.post(TOKEN_URL, headers=headers, data=data)
print(auth_response.json()) # returns a dictionary
print(auth_response.json()['access_token'])

{'access_token': 'BQAddq7hU1LVjh90WxhqYfmS2NqgpUTyb2LVDigcsT4gRSl4trZvOr_qxJUHuiFD0ChTsZ2tx4FvlsbuoWXtIPUXse0TGbjm0TWVrRJWh8Zo-hXuxpM81909fkt7Omn3-4Ie8fDRnyE', 'token_type': 'Bearer', 'expires_in': 3600}
BQAddq7hU1LVjh90WxhqYfmS2NqgpUTyb2LVDigcsT4gRSl4trZvOr_qxJUHuiFD0ChTsZ2tx4FvlsbuoWXtIPUXse0TGbjm0TWVrRJWh8Zo-hXuxpM81909fkt7Omn3-4Ie8fDRnyE


In [5]:
# RECUPERER DATA EN NOUS AUTHENTIFIANT AVEC L'ACCESS TOKEN ex. info d'un artiste spécifique
## Définir le headers avec l'access_token
## Penser à utiliser f-string ou .format() pour introduire l'access_token défini ci-dessus
access_token = auth_response.json().get('access_token') # access token à mettre en variable d'env pour pas qu'un user public réutilise
print(access_token)

BQAddq7hU1LVjh90WxhqYfmS2NqgpUTyb2LVDigcsT4gRSl4trZvOr_qxJUHuiFD0ChTsZ2tx4FvlsbuoWXtIPUXse0TGbjm0TWVrRJWh8Zo-hXuxpM81909fkt7Omn3-4Ie8fDRnyE


In [6]:
## Requête GET pour obtenir la data sur l'artiste
### Ici on utilise du python propre i.e requests.get() 

# We make a GET request and store the response 
# Add the Content-Type header set to the application/x-www-form-urlencoded value.
headers = {
    'Authorization': 'Bearer {}'.format(access_token)
}

In [7]:
response_artist = requests.get(BASE_URL + 'artists/4Z8W4fKeB5YxbusRsdQVPb', headers=headers) 
if response_artist.status_code == 200: 
    print(response_artist.json()) # Inspect response by printing it 
else:
    print(response_artist.status_code)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'}, 'followers': {'href': None, 'total': 11491901}, 'genres': ['art rock', 'alternative rock'], 'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb', 'id': '4Z8W4fKeB5YxbusRsdQVPb', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba03696716c9ee605006047fd', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174a03696716c9ee605006047fd', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178a03696716c9ee605006047fd', 'height': 160, 'width': 160}], 'name': 'Radiohead', 'popularity': 83, 'type': 'artist', 'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}


# 2. Get artist's nb of followers & genre

## 2.1. For one specific artist for whom we have the specific url

In [8]:
response_artist.json().get('followers')

{'href': None, 'total': 11491901}

In [9]:
followers = response_artist.json().get('followers')['total']
followers

11491901

In [10]:
name = response_artist.json().get('name')
name

'Radiohead'

In [11]:
genres = response_artist.json().get('genres')
print(genres), print(len(genres))

['art rock', 'alternative rock']
2


(None, None)

In [12]:
print(f"nb of followers for {name} = {followers}")

nb of followers for Radiohead = 11491901


## 2.2. For one specific artist for whom we have the name only (but not its spotify id)

### Retrieve Spotify ID for those artists

In [13]:
# Retrieve Spotify ID (see definition of Spotify ID: https://developer.spotify.com/documentation/web-api/concepts/spotify-uris-ids)
# Inspiration: https://stackoverflow.com/questions/41566971/how-to-get-spotify-artist-id-for-the-spotify-endpoint-url
response_artist_search = requests.get(BASE_URL + 'search?q=Daft+Punk&type=artist', headers=headers) 
if response_artist.status_code == 200: 
    print(response_artist_search.json()) # Inspect response by printing it 
else:
    print(response_artist_search.status_code)

{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=20&query=Daft%20Punk&type=artist', 'limit': 20, 'next': 'https://api.spotify.com/v1/search?offset=20&limit=20&query=Daft%20Punk&type=artist', 'offset': 0, 'previous': None, 'total': 815, 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'}, 'followers': {'href': None, 'total': 10473934}, 'genres': ['french house', 'electronic', 'electro'], 'href': 'https://api.spotify.com/v1/artists/4tZwfgrHOc3mvqYlEYSvVi', 'id': '4tZwfgrHOc3mvqYlEYSvVi', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba7bfd7835b5c1eee0c95fa6e', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174a7bfd7835b5c1eee0c95fa6e', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178a7bfd7835b5c1eee0c95fa6e', 'height': 160, 'width': 160}], 'name': 'Daft Punk', 'popularity': 80, 'type': 'artist', 'uri': 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'}, {'ext

In [14]:
# Version with name of artist defined first
artist_name= 'Daft Punk'
url_artist=artist_name.replace(' ','+')
'search?q='+url_artist+'&type=artist'

'search?q=Daft+Punk&type=artist'

In [15]:
response_artist_search = requests.get(BASE_URL + 'search?q='+url_artist+'&type=artist', headers=headers) 
if response_artist.status_code == 200: 
    print(response_artist_search.json()) # Inspect response by printing it 
else:
    print(response_artist_search.status_code)

{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=20&query=Daft%20Punk&type=artist', 'limit': 20, 'next': 'https://api.spotify.com/v1/search?offset=20&limit=20&query=Daft%20Punk&type=artist', 'offset': 0, 'previous': None, 'total': 815, 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'}, 'followers': {'href': None, 'total': 10473934}, 'genres': ['french house', 'electronic', 'electro'], 'href': 'https://api.spotify.com/v1/artists/4tZwfgrHOc3mvqYlEYSvVi', 'id': '4tZwfgrHOc3mvqYlEYSvVi', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba7bfd7835b5c1eee0c95fa6e', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174a7bfd7835b5c1eee0c95fa6e', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178a7bfd7835b5c1eee0c95fa6e', 'height': 160, 'width': 160}], 'name': 'Daft Punk', 'popularity': 80, 'type': 'artist', 'uri': 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'}, {'ext

In [16]:
# get artist's spotify id from key 'artists'
response_artist_search.json().get('artists')

{'href': 'https://api.spotify.com/v1/search?offset=0&limit=20&query=Daft%20Punk&type=artist',
 'limit': 20,
 'next': 'https://api.spotify.com/v1/search?offset=20&limit=20&query=Daft%20Punk&type=artist',
 'offset': 0,
 'previous': None,
 'total': 815,
 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'},
   'followers': {'href': None, 'total': 10473934},
   'genres': ['french house', 'electronic', 'electro'],
   'href': 'https://api.spotify.com/v1/artists/4tZwfgrHOc3mvqYlEYSvVi',
   'id': '4tZwfgrHOc3mvqYlEYSvVi',
   'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba7bfd7835b5c1eee0c95fa6e',
     'height': 640,
     'width': 640},
    {'url': 'https://i.scdn.co/image/ab67616100005174a7bfd7835b5c1eee0c95fa6e',
     'height': 320,
     'width': 320},
    {'url': 'https://i.scdn.co/image/ab6761610000f178a7bfd7835b5c1eee0c95fa6e',
     'height': 160,
     'width': 160}],
   'name': 'Daft Punk',
   'popularity': 80,
   'type': 'artist'

In [17]:
response_artist_search.json().get('artists')['items'][0]['id']

'4tZwfgrHOc3mvqYlEYSvVi'

### Use it to get nb of followers and genres

In [18]:
artist_id = response_artist_search.json().get('artists')['items'][0]['id']

response_artist_searched = requests.get(BASE_URL + 'artists/'+artist_id, headers=headers) 
if response_artist_searched.status_code == 200: 
    print(response_artist_searched.json()) # Inspect response by printing it 
else:
    print(response_artist_searched.status_code)


{'external_urls': {'spotify': 'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'}, 'followers': {'href': None, 'total': 10473934}, 'genres': ['french house', 'electronic', 'electro'], 'href': 'https://api.spotify.com/v1/artists/4tZwfgrHOc3mvqYlEYSvVi', 'id': '4tZwfgrHOc3mvqYlEYSvVi', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eba7bfd7835b5c1eee0c95fa6e', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab67616100005174a7bfd7835b5c1eee0c95fa6e', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f178a7bfd7835b5c1eee0c95fa6e', 'height': 160, 'width': 160}], 'name': 'Daft Punk', 'popularity': 80, 'type': 'artist', 'uri': 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'}


In [19]:
followers_2 = response_artist_searched.json().get('followers')['total']
name_2 = response_artist_searched.json().get('name')
genres_2 = response_artist.json().get('genres')
print(f"nb of followers for {name_2} = {followers_2}\n {len(genres_2)} Genre(s) associated = {genres_2}")

nb of followers for Daft Punk = 10473934
 2 Genre(s) associated = ['art rock', 'alternative rock']


## 2.3. For a list of artists for whom we have got the names only

### a. First get the list of artist: for example, an extract of 2023 Most streamed songs

In [20]:
filepath = os.path.join('..','data','2023_Most_Streamed_clean.csv')
df = pd.read_csv(filepath)
df.head()

,Unnamed: 0,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,released_date,rank_streams
0,0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,...,Major,80,89,83,31,0,8,4,2023-07-14,0
1,1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,...,Major,71,61,74,7,0,10,4,2023-03-23,1
2,2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,...,Major,51,32,53,17,0,31,6,2023-06-30,2
3,3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,...,Major,55,58,72,11,0,11,15,2019-08-23,3
4,4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,...,Minor,65,23,80,14,63,11,6,2023-05-18,4


In [21]:
# We only keep the artist column and the 5 first artists
df = df[['artist(s)_name','artist_count','streams']]
df = df.iloc[:5]
df

,artist(s)_name,artist_count,streams
0,"Latto, Jung Kook",2,141381703
1,Myke Towers,1,133716286
2,Olivia Rodrigo,1,140003974
3,Taylor Swift,1,800840817
4,Bad Bunny,1,303236322


In [22]:
# We get the information artist by artist esp. the main artist
def f(row):
    if row['artist_count'] == 1:
        val = row['artist(s)_name']
    else:
        i = row['artist(s)_name'].find(',')
        val = row['artist(s)_name'][:i]
    return val

f(df.iloc[0])

'Latto'

In [23]:
def g(row):
    if row['artist_count'] == 1:
        val = ""
    else:
        i = row['artist(s)_name'].find(',')
        val = row['artist(s)_name'][i+2:]
    return val

g(df.iloc[0])

'Jung Kook'

In [24]:
df['artist_1']=df.apply(f, axis=1)
df['other artists']=df.apply(g, axis=1)
df

,artist(s)_name,artist_count,streams,artist_1,other artists
0,"Latto, Jung Kook",2,141381703,Latto,Jung Kook
1,Myke Towers,1,133716286,Myke Towers,
2,Olivia Rodrigo,1,140003974,Olivia Rodrigo,
3,Taylor Swift,1,800840817,Taylor Swift,
4,Bad Bunny,1,303236322,Bad Bunny,


### b. Now retrieve Spotify ID for those artists & get nb of followers 

In [25]:
def fetch_followers(row):
    artist_name= row['artist_1']
    url_artist=artist_name.replace(' ','+')
    response_artist_search = requests.get(BASE_URL + 'search?q='+url_artist+'&type=artist', headers=headers) 
    
    if response_artist_search.status_code == 200: 
        artist_id = response_artist_search.json().get('artists')['items'][0]['id']
        response_artist_found = requests.get(BASE_URL + 'artists/'+artist_id, headers=headers) 
        if response_artist_found.status_code == 200: 
            followers = response_artist_found.json().get('followers')['total'] 
            return followers
        else:
            return "Error: "+ str(response_artist_found.status_code)
    else:
        return "Error: "+ str(response_artist_search.status_code)


fetch_followers(df.iloc[0])

1935470

In [26]:
df['spotify_followers_1_feb25']=df.apply(fetch_followers, axis=1)
df

,artist(s)_name,artist_count,streams,artist_1,other artists,spotify_followers_1_feb25
0,"Latto, Jung Kook",2,141381703,Latto,Jung Kook,1935470
1,Myke Towers,1,133716286,Myke Towers,,18732430
2,Olivia Rodrigo,1,140003974,Olivia Rodrigo,,43771431
3,Taylor Swift,1,800840817,Taylor Swift,,132558401
4,Bad Bunny,1,303236322,Bad Bunny,,91028503


### c. Also get genres

In [27]:
def fetch_genres(row):
    artist_name= row['artist_1']
    url_artist=artist_name.replace(' ','+')
    response_artist_search = requests.get(BASE_URL + 'search?q='+url_artist+'&type=artist', headers=headers) 
    
    if response_artist_search.status_code == 200: 
        artist_id = response_artist_search.json().get('artists')['items'][0]['id']
        response_artist_found = requests.get(BASE_URL + 'artists/'+artist_id, headers=headers) 
        if response_artist_found.status_code == 200: 
            genres = response_artist_found.json().get('genres')
            return genres
        else:
            return "Error: "+ str(response_artist_found.status_code)
    else:
        return "Error: "+ str(response_artist_search.status_code)


fetch_genres(df.iloc[0])

[]

In [28]:
df['artist_1_genres']=df.apply(fetch_genres, axis=1)
df
# a lot of artists now no longer have genres: seems to be a recent change with no communication by spotify (see https://community.spotify.com/t5/Spotify-for-Developers/Genres-from-artists-have-been-removed/td-p/6659559)

,artist(s)_name,artist_count,streams,artist_1,other artists,spotify_followers_1_feb25,artist_1_genres
0,"Latto, Jung Kook",2,141381703,Latto,Jung Kook,1935470,[]
1,Myke Towers,1,133716286,Myke Towers,,18732430,"[reggaeton, trap latino, urbano latino]"
2,Olivia Rodrigo,1,140003974,Olivia Rodrigo,,43771431,[]
3,Taylor Swift,1,800840817,Taylor Swift,,132558401,[]
4,Bad Bunny,1,303236322,Bad Bunny,,91028503,"[reggaeton, trap latino, latin, urbano latino]"


### d. Get artist's popularity

In [29]:
# Definition: The popularity of the artist. (https://developer.spotify.com/documentation/web-api/reference/get-an-artist)
# The value will be between 0 and 100, with 100 being the most popular. The artist's popularity is calculated from the popularity of all the artist's tracks.
def fetch_artist_popularity(row):
    artist_name= row['artist_1']
    url_artist=artist_name.replace(' ','+')
    response_artist_search = requests.get(BASE_URL + 'search?q='+url_artist+'&type=artist', headers=headers) 
    
    if response_artist_search.status_code == 200: 
        artist_id = response_artist_search.json().get('artists')['items'][0]['id']
        response_artist_found = requests.get(BASE_URL + 'artists/'+artist_id, headers=headers) 
        if response_artist_found.status_code == 200: 
            genres = response_artist_found.json().get('popularity')
            return genres
        else:
            return "Error: "+ str(response_artist_found.status_code)
    else:
        return "Error: "+ str(response_artist_search.status_code)


fetch_artist_popularity(df.iloc[0])

78

In [30]:
df['artist_1_popularity_feb25']=df.apply(fetch_artist_popularity, axis=1)
df

,artist(s)_name,artist_count,streams,artist_1,other artists,spotify_followers_1_feb25,artist_1_genres,artist_1_popularity_feb25
0,"Latto, Jung Kook",2,141381703,Latto,Jung Kook,1935470,[],78
1,Myke Towers,1,133716286,Myke Towers,,18732430,"[reggaeton, trap latino, urbano latino]",87
2,Olivia Rodrigo,1,140003974,Olivia Rodrigo,,43771431,[],86
3,Taylor Swift,1,800840817,Taylor Swift,,132558401,[],96
4,Bad Bunny,1,303236322,Bad Bunny,,91028503,"[reggaeton, trap latino, latin, urbano latino]",100


# 3. Get audio features for a list of tracks

Fields to retrieve: 
- audio-features (dict):
    - acousticness
    - danceability
    - duration
    - energy
    - key
    - instrumentalness
    - liveness
    - loudness
    - mode
    - speechiness
    - tempo (bpm)
    - valence
- audio analysis
    - tracks / time_signature (nb of beats per bar)
    - tracks / time_signature_confidence
    - segments / pitches
    - segments / timbre
- available_markets (list of strings)

### Search track id by inputting track name AND artist name (there might be several tracks with same title)

In [31]:
# use search endpoint: https://developer.spotify.com/documentation/web-api/reference/search
response_track_id = requests.get(BASE_URL + 'search?q=track:Be+The+One%20artist:Dua+Lipa&type=track&limit=10', headers=headers) 
if response_track_id.status_code == 200: 
    print(response_track_id.json()) # Inspect response by printing it 
else:
    print(response_track_id.status_code)

{'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=10&query=track%3ABe%20The%20One%20artist%3ADua%20Lipa&type=track', 'limit': 10, 'next': 'https://api.spotify.com/v1/search?offset=10&limit=10&query=track%3ABe%20The%20One%20artist%3ADua%20Lipa&type=track', 'offset': 0, 'previous': None, 'total': 95, 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6M2wZ9GZgrQXHCFfjv46we'}, 'href': 'https://api.spotify.com/v1/artists/6M2wZ9GZgrQXHCFfjv46we', 'id': '6M2wZ9GZgrQXHCFfjv46we', 'name': 'Dua Lipa', 'type': 'artist', 'uri': 'spotify:artist:6M2wZ9GZgrQXHCFfjv46we'}], 'available_markets': ['AR', 'AU', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'TW', 'TR', 'UY', 'US', 'AD', 'LI', 'MC', 'ID', 

In [32]:
dico = dict(response_track_id.json())
len(dico)

1

In [33]:
dico

{'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=10&query=track%3ABe%20The%20One%20artist%3ADua%20Lipa&type=track',
  'limit': 10,
  'next': 'https://api.spotify.com/v1/search?offset=10&limit=10&query=track%3ABe%20The%20One%20artist%3ADua%20Lipa&type=track',
  'offset': 0,
  'previous': None,
  'total': 95,
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6M2wZ9GZgrQXHCFfjv46we'},
       'href': 'https://api.spotify.com/v1/artists/6M2wZ9GZgrQXHCFfjv46we',
       'id': '6M2wZ9GZgrQXHCFfjv46we',
       'name': 'Dua Lipa',
       'type': 'artist',
       'uri': 'spotify:artist:6M2wZ9GZgrQXHCFfjv46we'}],
     'available_markets': ['AR',
      'AU',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK

In [34]:
# Test track_id
response_track = requests.get(BASE_URL + 'tracks/7FCfMXYTIiQ9b4hDYs4Iol', headers=headers) 
if response_track.status_code == 200: 
    print(response_track.json()) # Inspect response by printing it 
else:
    print(response_track.status_code)

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6M2wZ9GZgrQXHCFfjv46we'}, 'href': 'https://api.spotify.com/v1/artists/6M2wZ9GZgrQXHCFfjv46we', 'id': '6M2wZ9GZgrQXHCFfjv46we', 'name': 'Dua Lipa', 'type': 'artist', 'uri': 'spotify:artist:6M2wZ9GZgrQXHCFfjv46we'}], 'available_markets': ['AR', 'AU', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'TW', 'TR', 'UY', 'US', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'DM', 'FJ', 'GM', 'GE', 'GD', 

In [35]:
# Test track_id (another id????)
# response_track = requests.get(BASE_URL + 'tracks/5nt8byg9jjRN24oy7I61wC', headers=headers) 
# if response_track.status_code == 200: 
#     print(response_track.json()) # Inspect response by printing it 
# else:
#     print(response_track.status_code)

In [36]:
# Retrieve track_id from json
response_track_id.json().get('tracks')['items'][0]['id']

'7FCfMXYTIiQ9b4hDYs4Iol'

In [37]:
# get available markets
markets = response_track.json().get('available_markets')
len(markets), markets

(179,
 ['AR',
  'AU',
  'BE',
  'BO',
  'BR',
  'BG',
  'CA',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DK',
  'DO',
  'EC',
  'EE',
  'SV',
  'FI',
  'FR',
  'GR',
  'GT',
  'HN',
  'HK',
  'HU',
  'IS',
  'IT',
  'LV',
  'LT',
  'LU',
  'MY',
  'MT',
  'MX',
  'NL',
  'NZ',
  'NI',
  'NO',
  'PA',
  'PY',
  'PE',
  'PH',
  'PL',
  'PT',
  'SG',
  'SK',
  'ES',
  'SE',
  'TW',
  'TR',
  'UY',
  'US',
  'AD',
  'LI',
  'MC',
  'ID',
  'JP',
  'TH',
  'VN',
  'RO',
  'IL',
  'ZA',
  'SA',
  'AE',
  'BH',
  'QA',
  'OM',
  'KW',
  'EG',
  'MA',
  'DZ',
  'TN',
  'LB',
  'JO',
  'PS',
  'IN',
  'BY',
  'KZ',
  'MD',
  'UA',
  'AL',
  'BA',
  'HR',
  'ME',
  'MK',
  'RS',
  'SI',
  'KR',
  'BD',
  'PK',
  'LK',
  'GH',
  'KE',
  'NG',
  'TZ',
  'UG',
  'AG',
  'AM',
  'BS',
  'BB',
  'BZ',
  'BT',
  'BW',
  'BF',
  'CV',
  'CW',
  'DM',
  'FJ',
  'GM',
  'GE',
  'GD',
  'GW',
  'GY',
  'HT',
  'JM',
  'KI',
  'LS',
  'LR',
  'MW',
  'MV',
  'ML',
  'MH',
  'FM',
  'NA',
  'NR',
  'NE',
  

In [38]:
# ⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️ TO DEBUG: ERROR 403 - Bad OAuth request (wrong consumer key, bad nonce, expired timestamp...). Unfortunately, re-authenticating the user won't help here.⚠️⚠️

track_id = response_track_id.json().get('tracks')['items'][0]['id']

response_track_audio_features = requests.get(BASE_URL + 'audio-features/'+track_id, headers=headers) 
if response_track_audio_features.status_code == 200: 
    print(response_track_audio_features.json()) # Inspect response by printing it 
else:
    print(response_track_audio_features.status_code)

403


In [39]:
# test with spotify's example => same error !!!!!
response_track_audio_features = requests.get(BASE_URL + 'audio-features/11dFghVXANMlKmJXsNCbNl', headers=headers) 
if response_track_audio_features.status_code == 200: 
    print(response_track_audio_features.json()) # Inspect response by printing it 
else:
    print(response_track_audio_features.status_code)

403


### For our actual dataset 2024

In [40]:
# Upload the "final" dataset we will use i.e Most Streamed 2024 songs
filepath_2024 = os.path.join('..','inputs','2024_Most_Streamed_Spotify_Songs.csv')
df = pd.read_csv(filepath_2024, encoding= 'unicode_escape')
df.head()

,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",...,684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0
1,Not Like Us,Not Like Us,Kendrick Lamar,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",...,3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1
2,i like the way you kiss me,I like the way you kiss me,Artemas,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",...,536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0
3,Flowers,Flowers - Single,Miley Cyrus,1/12/2023,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",...,"2,182",264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0
4,Houdini,Houdini,Eminem,5/31/2024,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",...,1,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1


In [41]:
df.columns

Index(['Track', 'Album Name', 'Artist', 'Release Date', 'ISRC',
       'All Time Rank', 'Track Score', 'Spotify Streams',
       'Spotify Playlist Count', 'Spotify Playlist Reach',
       'Spotify Popularity', 'YouTube Views', 'YouTube Likes', 'TikTok Posts',
       'TikTok Likes', 'TikTok Views', 'YouTube Playlist Reach',
       'Apple Music Playlist Count', 'AirPlay Spins', 'SiriusXM Spins',
       'Deezer Playlist Count', 'Deezer Playlist Reach',
       'Amazon Playlist Count', 'Pandora Streams', 'Pandora Track Stations',
       'Soundcloud Streams', 'Shazam Counts', 'TIDAL Popularity',
       'Explicit Track'],
      dtype='object')

In [43]:
def fetch_markets(row):
    artist_name= row['Artist']
    url_artist=artist_name.replace(' ','+')
    url_track=row['Track'].replace(' ','+')

    response_track_search = requests.get(BASE_URL + 'search?q=track:'+url_track+'%20artist:'+url_artist+'&type=track&limit=10', headers=headers) 
    
    if response_track_search.status_code == 200: 
        track_id = response_track_search.json().get('tracks')['items'][0]['id']
        response_track_found = requests.get(BASE_URL + 'tracks/'+track_id, headers=headers) 
        if response_track_found.status_code == 200: 
            markets = response_track_found.json().get('available_markets')
            return markets
        else:
            return "Error: "+ str(response_track_found.status_code)
    else:
        return "Error: "+ str(response_track_search.status_code)


fetch_markets(df.iloc[0])

['AR',
 'AU',
 'AT',
 'BE',
 'BO',
 'BR',
 'BG',
 'CA',
 'CL',
 'CO',
 'CR',
 'CY',
 'CZ',
 'DK',
 'DO',
 'DE',
 'EC',
 'EE',
 'SV',
 'FI',
 'FR',
 'GR',
 'GT',
 'HN',
 'HK',
 'HU',
 'IS',
 'IE',
 'IT',
 'LV',
 'LT',
 'LU',
 'MY',
 'MT',
 'MX',
 'NL',
 'NZ',
 'NI',
 'NO',
 'PA',
 'PY',
 'PE',
 'PH',
 'PL',
 'PT',
 'SG',
 'SK',
 'ES',
 'SE',
 'CH',
 'TW',
 'TR',
 'UY',
 'US',
 'GB',
 'AD',
 'LI',
 'MC',
 'ID',
 'JP',
 'TH',
 'VN',
 'RO',
 'IL',
 'ZA',
 'SA',
 'AE',
 'BH',
 'QA',
 'OM',
 'KW',
 'EG',
 'MA',
 'DZ',
 'TN',
 'LB',
 'JO',
 'PS',
 'IN',
 'KZ',
 'MD',
 'UA',
 'AL',
 'BA',
 'HR',
 'ME',
 'MK',
 'RS',
 'SI',
 'KR',
 'BD',
 'PK',
 'LK',
 'GH',
 'KE',
 'NG',
 'TZ',
 'UG',
 'AG',
 'AM',
 'BS',
 'BB',
 'BZ',
 'BT',
 'BW',
 'BF',
 'CV',
 'CW',
 'DM',
 'FJ',
 'GM',
 'GE',
 'GD',
 'GW',
 'GY',
 'HT',
 'JM',
 'KI',
 'LS',
 'LR',
 'MW',
 'MV',
 'ML',
 'MH',
 'FM',
 'NA',
 'NR',
 'NE',
 'PW',
 'PG',
 'WS',
 'SM',
 'ST',
 'SN',
 'SC',
 'SL',
 'SB',
 'KN',
 'LC',
 'VC',
 'SR',
 'TL',
 'TO',

In [45]:
# For the first 5 rows only
df['track_markets']=df.iloc[:5].apply(fetch_markets, axis=1)
df

,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track,track_markets
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",...,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
1,Not Like Us,Not Like Us,Kendrick Lamar,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",...,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
2,i like the way you kiss me,I like the way you kiss me,Artemas,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",...,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
3,Flowers,Flowers - Single,Miley Cyrus,1/12/2023,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",...,264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
4,Houdini,Houdini,Eminem,5/31/2024,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",...,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,For the Last Time,For the Last Time,$uicideboy$,9/5/2017,QM8DG1703420,"4,585",19.4,"305,049,963","65,770","5,103,054",...,2.0,"14,217",NaN,"20,104,066","13,184","50,633,006","656,337",NaN,1,NaN
4596,Dil Meri Na Sune,"Dil Meri Na Sune (From ""Genius"")",Atif Aslam,7/27/2018,INT101800122,"4,575",19.4,"52,282,360","4,602","1,449,767",...,1.0,927,NaN,NaN,NaN,NaN,"193,590",NaN,0,NaN
4597,Grace (feat. 42 Dugg),My Turn,Lil Baby,2/28/2020,USUG12000043,"4,571",19.4,"189,972,685","72,066","6,704,802",...,1.0,74,6.0,"84,426,740","28,999",NaN,"1,135,998",NaN,1,NaN
4598,Nashe Si Chadh Gayi,November Top 10 Songs,Arijit Singh,11/8/2016,INY091600067,"4,591",19.4,"145,467,020","14,037","7,387,064",...,NaN,NaN,7.0,"6,817,840",NaN,NaN,"448,292",NaN,0,NaN
